In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [4]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kacho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
 print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [6]:
## DATA PROCESSING 

In [7]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv' ,encoding = 'ISO-8859-1')

In [8]:
df.shape

(1599999, 6)

In [9]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
#columns name is missing 

In [11]:
column_names = ['target','id','data','flag','user','text']
df = pd.read_csv('training.1600000.processed.noemoticon.csv' ,names=column_names,encoding = 'ISO-8859-1')

In [12]:
df.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
 df.shape

(1600000, 6)

In [14]:
df.isnull().sum()  

target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

In [15]:
#checking distribution of taret column 
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [16]:
##balanced dataset
# target from 4 to 1
# 0 for negative text and 1 for positive text

In [17]:
df.replace({'target':{4:1}}, inplace= True)

In [18]:
df.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [19]:
# Stemming

In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [22]:
import pandas as pd
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stem_text(text):
    return " ".join(stemmer.stem(word) for word in text.split())

chunk_size = 100000
results = []
for chunk in pd.read_csv(
    'training.1600000.processed.noemoticon.csv',
    encoding='ISO-8859-1',
    names=["target", "ids", "date", "flag", "user", "text"],
    chunksize=chunk_size,
    # on_bad_lines='skip',
    # engine='python'
):
    chunk["stemmed"] = chunk["text"].astype(str).apply(stem_text)
    results.append(chunk)

final_df = pd.concat(results, ignore_index=True)
final_df.to_csv("stemmed_output.csv", index=False)


In [30]:
final_df.head()

,target,ids,date,flag,user,text,stemmed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't updat hi facebook by te...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan i dive mani time for the ball. manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it' not behav at all. i'm..."


In [36]:
# separating data and labels
df = final_df
df.replace({'target':{4:1}}, inplace= True)

In [37]:
X = df['stemmed'].values

In [38]:
Y = df['target'].values

In [39]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - awww, that' a bummer. you shoulda got david carr of third day to do it. ;d"
 "is upset that he can't updat hi facebook by text it... and might cri as a result school today also. blah!"
 '@kenichan i dive mani time for the ball. manag to save 50% the rest go out of bound'
 ... 'are you readi for your mojo makeover? ask me for detail'
 'happi 38th birthday to my boo of alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [40]:
print(Y)

[0 0 0 ... 1 1 1]


In [41]:
# splitting data to test data and training data
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,stratify =Y,random_state = 2)

In [42]:
print(X.shape,X_train.shape,X_test.shape)
print(Y.shape,Y_train.shape,Y_test.shape)


(1600000,) (1280000,) (320000,)
(1600000,) (1280000,) (320000,)


In [43]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [44]:
# training the model
# Logistic Regression
model = LogisticRegression(max_iter=1000)

In [45]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [46]:
# model evaluation
# Accuracy score

In [47]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)
print('Accuracy score on the training data: ', training_data_accuracy)

Accuracy score on the training data:  0.816290625


In [48]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)
print('Accuracy score on the test data: ', test_data_accuracy)

Accuracy score on the test data:  0.79744375


In [49]:
# saving the trained model

In [50]:
import pickle

In [51]:
file = 'trained_model.sav'
pickle.dump(model,open(file,'wb'))

In [52]:
#using the saved model for future prediction

In [53]:
X_new = X_test[100]
print(Y_test[100])
prediction = model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
    print('Negative tweet')
else:
    print('positive Tweet')
    


0
[0]
Negative tweet
